# Westeros Tutorial - Adding representation of renewables (part1/3): Introducing `"firm capacity"`

This tutorial demonstrates how to apply various features of the MESSAGEix formulation to provide a more realistic representation of renewable energy integration in the energy system.

In the first part of this series of tutorials, we show how to ensure that in a model setup which does not use sub-annual timesteps, and therefore does not depict peak load, we can still account for [`"firm capacity"`](https://docs.messageix.org/en/stable/model/MESSAGE/model_core.html?highlight=FIRM_CAPACITY_PROVISION#equation-firm-capacity-provision). In power systems, firm or reliable capacity means sufficient power capacity from different electricity generation plants is available to maintain system reliability through reasonable load and contingency events.

Further information can be found in [Sullivan et al., 2013](https://doi.org/10.1016/j.esr.2013.01.001).

**Pre-requisites**
- You have the *MESSAGEix* framework installed and working
- You have run Westeros scenario which adds emission bounds (``westeros_emissions_bounds.ipynb``) and solved it successfully

In [ ]:
import ixmp

mp = ixmp.Platform()

## Load an existing scenario

We load the existing scenario '*emission_bound*' and clone it to a new scenario called '*firm_capacity*'. We apply the `"firm_capacity"` constraints in this new scenario.

In [ ]:
from message_ix import Scenario

model = "Westeros Electrified"
base = Scenario(mp, model=model, scenario="emission_bound")
scen = base.clone(
    model=model,
    scenario="firm_capacity",
    annotation="illustration of firm-capacity formulation",
    keep_solution=False,
)
scen.check_out()

## Retrieve required parameters for modification

In [ ]:
model_horizon = scen.set("year")
country = "Westeros"

## `"peak_load_factor"` - Describing the Electricity Sector
For systems across the U.S., load duration curves are similar from one location to another. Based on load data it was calculated that the peak load is, on average, 1.7 times the average load (a load factor of 59%). As illustrated in the figure below, when coupled with a standard reserve margin of 15–20% of peak load, firm capacity requirements for the U.S. should be roughly twice average load. ([Sullivan et al., 2013](https://doi.org/10.1016/j.esr.2013.01.001))

<img src='_static/firm-capacity.png' width='400'>

In this example, a [`"peak_load_factor"`](https://docs.messageix.org/en/stable/model/MESSAGE/parameter_def.html?highlight=peak_load_factor#parameters-of-the-demand-section) of 2 is introduced for the `"level": "secondary"` and `"commodity": "electricity"`, which is the level/commodity where power plant generation is accounted for.

In [ ]:
import pandas as pd

peak_load_factor = pd.DataFrame(
    {
        "node": country,
        "commodity": "electricity",
        "level": "secondary",
        "year": model_horizon,
        "time": "year",
        "value": 2,
        "unit": "-",
    }
)
scen.add_par("peak_load_factor", peak_load_factor)

## `rating_bin` and `reliability_factor` - Describing the power plant reliability
### Ensuring enough firm capacity
Toward meeting the firm capacity requirement, conventional generating technologies contribute their nameplate generation capacity (see figure above). 

This therefore implies that for conventional power plants i.e., for `"coal_ppl"`, a `"reliability_factor"` of 1 defines the `"rating_bin"` (the maximum share of technology in commodity use per rating) with [`rating="firm"`](https://docs.messageix.org/en/stable/model/MESSAGE/parameter_def.html?highlight=rating_bin#input-output-mapping-costs-and-engineering-specifications). This is predefined as part of the model formulation.

In order to account for the integration challenge at higher shares of variable renewables, their contribution to the capacity requirements i.e., their capacity value, declines as the market share of the technology increases. 
We therefore assume, that up to a share of 20% of the total electricity supply, the capacity of `"wind_ppl"` adds by 80% to the firm capacity of the power system. This means `value=0.2` of the `"rating_bin"` and `value=0.8` of the `"reliability_factor"`, defined as `rating="r1"`. The remaining 80% of the installed capacity contribute to 5% to the firm capacity, and is defined as `rating="r2"`. 

In [ ]:
# Create the base dictionary for "rating_bin" and "reliability_factor" parameter
base_dict = dict(
    node=country,
    commodity="electricity",
    level="secondary",
    unit="-",
    time="year",
    year_act=model_horizon,
)

In [ ]:
from message_ix import make_df

# Add the ratings as a set
scen.add_set("rating", ["r1", "r2"])

# For the wind power plant
name = "rating_bin"

rating_bin = make_df(name, **base_dict, technology="wind_ppl", value=0.2, rating="r1")
scen.add_par(name, rating_bin)

rating_bin = make_df(name, **base_dict, technology="wind_ppl", value=0.8, rating="r2")
scen.add_par(name, rating_bin)

name = "reliability_factor"

reliability_factor = make_df(
    name, **base_dict, technology="wind_ppl", value=0.8, rating="r1"
)
scen.add_par(name, reliability_factor)

reliability_factor = make_df(
    name, **base_dict, technology="wind_ppl", value=0.05, rating="r2"
)
scen.add_par(name, reliability_factor)

# For the coal power plant
reliability_factor = make_df(
    name, **base_dict, technology="coal_ppl", value=1, rating="firm"
)
scen.add_par(name, reliability_factor)

### Commit and solve

In [ ]:
scen.commit(comment="define parameters for renewable implementation")
scen.set_as_default()

In [ ]:
scen.solve()

In [ ]:
scen.var("OBJ")["lvl"]

# Plotting Results

In [ ]:
from message_ix.report import Reporter
from message_ix.util.tutorial import prepare_plots

%matplotlib inline

rep_base = Reporter.from_scenario(base)
prepare_plots(rep_base)


rep_scen = Reporter.from_scenario(scen)
prepare_plots(rep_scen)

## Activity
***
When comparing the results of the original scenario without the reliability factor ('*emission_bound*') and the results of our newly modified scenario ('*firm_capacity*'), for the same carbon price we can observe that the activity of the `"wind_ppl"` and the `"coal_ppl"` is identical, which is what we hoped to achieve.

### Scenario: '*emission_bound*'

In [ ]:
rep_base.set_filters(t=["coal_ppl", "wind_ppl"])
rep_base.get("plot activity")

### Scenario: '*firm_capacity*'

In [ ]:
rep_scen.set_filters(t=["coal_ppl", "wind_ppl"])
rep_scen.get("plot activity")

## Capacity
***
The difference between the two scenarios can be seen when comparing the capacity. Whereas in the '*emission_bound*' scenario, coal capacity is quickly phased out, the `"firm_capacity"` scenario shows that there is a substantial amount of capacity required from the `"coal_ppl"` in order for the `"wind_ppl"` to achieve such high market share levels.

Please note the different maximum values of the y-axis of both plots.

### Scenario: '*emission_bound*'

In [ ]:
rep_base.get("plot capacity")

### Scenario: '*firm_capacity*'

In [ ]:
rep_scen.get("plot capacity")

## Prices
***
This has a substantial impact on the electricity price, which is reflected in the prices for lighting.

Please note the different maximum values of the y-axis of both plots.

### Scenario: '*emission_bound*'

In [ ]:
rep_base.set_filters(t=None, c=["light"])
rep_base.get("plot prices")

### Scenario: '*firm_capacity*'

In [ ]:
rep_scen.set_filters(t=None, c=["light"])
rep_scen.get("plot prices")

In [ ]:
mp.close_db()